In [1]:
"""A notebook to visualize audio embeddings of the HuggingFace Speech Commands dataset."""
%set_env PYTORCH_ENABLE_MPS_FALLBACK=1

import meerkat as mk

%load_ext autoreload
%autoreload 2

# Set your device here
device = "mps"

env: PYTORCH_ENABLE_MPS_FALLBACK=1


In [3]:
mk.gui.start(api_port=5010, frontend_port=8001, dev=False, skip_build=False)

(APIInfo(api=<fastapi.applications.FastAPI object at 0x13ae93d60>, port=5010, server=<meerkat.interactive.server.Server object at 0x103bde3d0>, name='127.0.0.1', shared=False, process=None, _url=None),
 FrontendInfo(package_manager='npm', port=8003, name='localhost', shared=False, process=<Popen: returncode: None args: ['python', '-m', 'http.server', '8003']>, _url=None))

## Load the Dataset
In this demo, we will be working with [`music_genres_small`](https://huggingface.co/datasets/lewtun/music_genres_small) dataset on HuggingFace.

In [4]:
dataset = mk.get(name="lewtun/music_genres_small", registry="huggingface")

/Users/sabrieyuboglu/opt/miniconda3/envs/meerkat/lib/python3.9/site-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)


[03/11/23 20:45:39] WARNING  [_create_builder_config()] [datasets.builder: 463] :: Using custom data ]8;id=593159;file:///Users/sabrieyuboglu/opt/miniconda3/envs/meerkat/lib/python3.9/site-packages/datasets/builder.py\builder.py]8;;\:]8;id=759472;file:///Users/sabrieyuboglu/opt/miniconda3/envs/meerkat/lib/python3.9/site-packages/datasets/builder.py#463\463]8;;\
                             configuration lewtun--music_genres_small-2686d03f87ff3ace                             

                    WARNING  [download_and_prepare()] [datasets.builder: 641] :: Reusing dataset     ]8;id=774455;file:///Users/sabrieyuboglu/opt/miniconda3/envs/meerkat/lib/python3.9/site-packages/datasets/builder.py\builder.py]8;;\:]8;id=811242;file:///Users/sabrieyuboglu/opt/miniconda3/envs/meerkat/lib/python3.9/site-packages/datasets/builder.py#641\641]8;;\
                             parquet                                                                               
                             (/Users/sabrieyuboglu/.cache/huggingface/datasets/lewtun___parquet/lewt               
                             un--music_genres_small-2686d03f87ff3ace/0.0.0/2a3b91fbd88a2c90d1dbbb32b               
                             460cf621d31bd5b05b934492fdef7d8d6f236ec)                                              

  0%|          | 0/1 [00:00<?, ?it/s]

/Users/sabrieyuboglu/code/meerkat/meerkat/columns/scalar/arrow.py:205: UserWarning: Unable to check if column is a valid primary key: Function 'unique' has no kernel matching input types (struct<bytes: binary, path: string>)
  warnings.warn(f"Unable to check if column is a valid primary key: {e}")


In [5]:
from typing import Any, Dict
import datasets as hf_datasets

from meerkat.interactive.formatter import AudioFormatterGroup
from meerkat.cells.audio import Audio

df = dataset["train"].view()

# The audio column is a dictionary containing the bytes.
# Extract the bytes lazily.
# The byte string is actually the fastest way to display the audio,
# because the encoding is already done.
df["audio"] = df["audio"].defer(lambda x: x["bytes"])

# Set the formatter for this column.
df["audio"].formatters = AudioFormatterGroup()

In [6]:
df

## Encode the dataset with Wav2Vec2
Encode the dataset with Wav2Vec2. This will take a few minutes.

You can also optionally download the embeddings from huggingface. See the code for how to do this

In [7]:
# Make a column that returns the audio tensors @ sampling rage 16kHz.
from meerkat.cells.audio import Audio
import datasets as hf_datasets

sampling_rate = 16000

def to_mk_audio(audio: bytes) -> Audio:
    audio_dict = hf_datasets.Audio().decode_example({"path": None, "bytes": audio})
    return Audio(data=audio_dict["array"], sampling_rate=audio_dict["sampling_rate"])

def to_array(audio: Audio):
    return audio.resample(sampling_rate).data

df_embed = df[["song_id", "audio"]]
df_embed["audio"] = df_embed["audio"].defer(to_mk_audio)
df_embed["audio_tensor"] = df_embed["audio"].defer(to_array)

MemoryError: Cannot allocate write+execute memory for ffi.callback(). You might be running on a system that prevents this. For more information, see https://cffi.readthedocs.io/en/latest/using.html#callbacks

In [ ]:
import torch
from transformers import AutoProcessor, Wav2Vec2Model


processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base-960h", device=device)
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h").to(device)

In [ ]:
def embed(audio_tensor: torch.Tensor):
    audio_tensor = audio_tensor.type(torch.float32).to(device)
    inputs = processor(audio_tensor, sampling_rate=sampling_rate, return_tensors="pt", device=device)
    inputs["input_values"] = inputs["input_values"].to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    return last_hidden_states.mean(dim=1).squeeze().cpu()

df_embed["embeddings"] = df_embed["audio_tensor"].map(embed, use_ray=False, pbar=True)
df_embed["embeddings"] = df_embed["embeddings"].to("cpu")

In [8]:
# Optionally download embeddings from huggingface
df_embed = mk.DataFrame.read("https://huggingface.co/datasets/meerkat-ml/meerkat-dataframes/resolve/main/music_genres_small-wav2vec2-embedded.mk.tar.gz")

## Make the interface

In [15]:
plot_df = df.merge(df_embed, on="song_id")

In [16]:
# compute umap of embeddings
from umap import UMAP

umap = UMAP(n_components=2)
umap = umap.fit_transform(plot_df["embeddings"])
plot_df["umap_1"] = umap[:, 0]
plot_df["umap_2"] = umap[:, 1]


In [17]:
plot_df = plot_df.mark()
plot = mk.gui.plotly.ScatterPlot(df=plot_df, x="umap_1", y="umap_2",)

# Because we're using the reactive decorator, the filter function will re-run whenever
# plot.selected changes. This will update the gallery to only show the selected points.
@mk.gui.reactive
def filter(selected: list, df: mk.DataFrame):
    return df[df.primary_key.isin(selected)]

filtered_df = filter(plot.selected, plot_df)
table = mk.gui.Table(filtered_df)

mk.gui.html.flexcol(
    [plot, table],
    classes="h-[1200px]",
)